In [1]:
# import os
# import sys
# spark_home = os.environ.get('SPARK_HOME', None)
# if not spark_home:
#     raise ValueError('SPARK_HOME environment variable is not set')
# sys.path.insert(0, os.path.join(spark_home, 'python'))
# sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
# exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [1]:
import findspark
findspark.init()
import pyspark
import numpy

import atexit
from time import time, strftime, localtime
from datetime import timedelta

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

conf = pyspark.SparkConf()
conf.setAppName('My Jupyter Notebook')
conf.set("spark.cores.max", "16")
if 'sc' in locals():
    sc.stop()
sc = pyspark.SparkContext(conf=conf)
conf.set("spark.yarn.executor.memoryOverhead", "0")
conf.set("spark.yarn.executor.memory", "512M")
conf.set("spark.yarn.driver.memory", "512M")

conf.set("spark.submit.deployMode", "client")

In [3]:
def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%Y-%m-%d %H:%M:%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    print(line)
    print(secondsToStr(), '-', s)
    if elapsed:
        print("Elapsed time:", elapsed)
    print(line)
    print()

def endlog():
    end = time()
    elapsed = end-start
#     print("Elapsed time 2: ", elapsed)
    log("End Program", secondsToStr(elapsed))

In [4]:
def randomForestMLib(cores):
    start = time()
#     atexit.register(endlog)
    log("Start program: " + str(cores))
    
    data = MLUtils.loadLibSVMFile(sc, "/house_data")
    (trainingData, testData) = data.randomSplit([0.7, 0.3])
    
    model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)
    
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    
    testErr = labelsAndPredictions.filter(
        lambda lp: lp[0] != lp[1]).count() / float(testData.count())
    
    print('Test Error = ' + str(testErr))
    print('Learned classification forest model:')
    print(model.toDebugString())
    end = time()
    elapsed = end - start
    print("End program " + str(cores) + ": " + str(secondsToStr(elapsed)))
#     endlog()

In [5]:
randomForestMLib(2)

2018-02-28 18:39:03 - Start program: 2

Test Error = 0.06451612903225806
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 378 <= 36.0)
     If (feature 214 <= 253.0)
      Predict: 0.0
     Else (feature 214 > 253.0)
      Predict: 1.0
    Else (feature 378 > 36.0)
     Predict: 1.0
  Tree 1:
    If (feature 378 <= 0.0)
     If (feature 380 <= 0.0)
      Predict: 0.0
     Else (feature 380 > 0.0)
      Predict: 1.0
    Else (feature 378 > 0.0)
     If (feature 549 <= 251.0)
      Predict: 1.0
     Else (feature 549 > 251.0)
      If (feature 380 <= 0.0)
       Predict: 0.0
      Else (feature 380 > 0.0)
       Predict: 1.0
  Tree 2:
    If (feature 262 <= 0.0)
     If (feature 407 <= 0.0)
      Predict: 0.0
     Else (feature 407 > 0.0)
      Predict: 1.0
    Else (feature 262 > 0.0)
     If (feature 176 <= 208.0)
      If (feature 462 <= 0.0)
       Predict: 0.0
      Else (feature 462 > 0.0)
       Predict: 1.0
     Else (featu